In [3]:
! pip install ucimlrepo
! pip install numpy pandas scikit-learn

## Лабораторная работа 1


In [8]:
##############################
# 1. УСТАНОВКА/ИМПОРТ БИБЛИОТЕК
##############################

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, mean_squared_error, mean_absolute_error, r2_score)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Для загрузки Breast Cancer Wisconsin Dataset
from sklearn.datasets import load_breast_cancer

# Для загрузки Concrete Dataset
from ucimlrepo import fetch_ucirepo

##############################
# 1. ВЫБОР НАЧАЛЬНЫХ УСЛОВИЙ
##############################

# 1.a Breast Cancer Wisconsin Dataset (задача классификации)
data_bc = load_breast_cancer()
X_class = pd.DataFrame(data_bc.data, columns=data_bc.feature_names)
y_class = pd.Series(data_bc.target)
print("\n[INFO] Breast Cancer dataset loaded.")
print(f"Features shape: {X_class.shape}, Target shape: {y_class.shape}")

# 1.b Concrete Compressive Strength Dataset (задача регрессии)
concrete_compressive_strength = fetch_ucirepo(id=165)  # id=165 для Concrete
X_concrete = concrete_compressive_strength.data.features
y_concrete = concrete_compressive_strength.data.targets

print("\n[INFO] Concrete dataset metadata:")
print(concrete_compressive_strength.metadata)

##############################
# 2. СОЗДАНИЕ БЕЙЗЛАЙНА
##############################

# 2.1 КЛАССИФИКАЦИЯ (Breast Cancer Wisconsin)

print("\n=== КЛАССИФИКАЦИЯ (Breast Cancer) ===")
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Бейзлайн KNN Classifier (без тюнинга)
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(Xc_train, yc_train)
yc_pred = knn_classifier.predict(Xc_test)

acc_base = accuracy_score(yc_test, yc_pred)
prec_base = precision_score(yc_test, yc_pred, average='macro')
rec_base = recall_score(yc_test, yc_pred, average='macro')
f1_base = f1_score(yc_test, yc_pred, average='macro')

print("\n[Бейзлайн: KNN Classifier]")
print(f"Accuracy:  {acc_base:.4f}")
print(f"Precision: {prec_base:.4f}")
print(f"Recall:    {rec_base:.4f}")
print(f"F1-score:  {f1_base:.4f}")

# 2.2 РЕГРЕССИЯ (Concrete)

print("\n=== РЕГРЕССИЯ (Concrete) ===")
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_concrete, y_concrete, test_size=0.2, random_state=42
)

knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(Xr_train, yr_train)
yr_pred = knn_regressor.predict(Xr_test)

mse_base = mean_squared_error(yr_test, yr_pred)
r2_base = r2_score(yr_test, yr_pred)

print("\n[Бейзлайн: KNN Regressor]")
print(f"MSE:  {mse_base:.4f}")
print(f"R^2:  {r2_base:.4f}")

##############################
# 3. УЛУЧШЕНИЕ БЕЙЗЛАЙНА
##############################

# GridSearchCV для классификации
pipeline_class = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid_class = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_search_class = GridSearchCV(
    pipeline_class,
    param_grid_class,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)
grid_search_class.fit(Xc_train, yc_train)

print("\nЛучшие параметры (Breast Cancer Classification):", grid_search_class.best_params_)
best_clf = grid_search_class.best_estimator_

yc_pred_best = best_clf.predict(Xc_test)
acc_best = accuracy_score(yc_test, yc_pred_best)
f1_best = f1_score(yc_test, yc_pred_best, average='macro')

print("[Улучшенный KNN Classifier] на тесте:")
print(f"Accuracy: {acc_best:.4f}, F1-macro: {f1_best:.4f}")

# GridSearchCV для регрессии
pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

param_grid_reg = {
    'scaler': [StandardScaler(), MinMaxScaler(), None],
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_search_reg = GridSearchCV(
    pipeline_reg,
    param_grid_reg,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_search_reg.fit(Xr_train, yr_train)

print("\nЛучшие параметры (Concrete Regression):", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_

yr_pred_best = best_reg.predict(Xr_test)
mse_best = mean_squared_error(yr_test, yr_pred_best)
r2_best = r2_score(yr_test, yr_pred_best)

print("[Улучшенный KNN Regressor] на тесте:")
print(f"MSE: {mse_best:.4f}, R^2: {r2_best:.4f}")

##############################
# 4. ИМПЛЕМЕНТАЦИЯ KNN “С НУЛЯ”
##############################

class CustomKNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self

    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unknown metric.")

    def predict(self, X):
        preds = []
        X = np.array(X)
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            neighbors_idx = np.argsort(distances)[:self.n_neighbors]
            neighbors_labels = self.y_train[neighbors_idx]
            values, counts = np.unique(neighbors_labels, return_counts=True)
            preds.append(values[np.argmax(counts)])
        return np.array(preds)


class CustomKNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self

    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unknown metric.")

    def predict(self, X):
        preds = []
        X = np.array(X)
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            neighbors_idx = np.argsort(distances)[:self.n_neighbors]
            neighbors_values = self.y_train[neighbors_idx]
            preds.append(np.mean(neighbors_values))
        return np.array(preds)

# 4.a Кастомная модель (классификация)
print("\n=== Custom KNN Classifier ===")
custom_clf = CustomKNNClassifier(n_neighbors=5, metric='euclidean')
custom_clf.fit(Xc_train, yc_train)
yc_pred_custom = custom_clf.predict(Xc_test)

acc_custom = accuracy_score(yc_test, yc_pred_custom)
f1_custom = f1_score(yc_test, yc_pred_custom, average='macro')
print(f"[Custom KNN Classifier] Accuracy: {acc_custom:.4f}, F1-macro: {f1_custom:.4f}")

# 4.b Кастомная модель (регрессия)
print("\n=== Custom KNN Regressor ===")
custom_reg = CustomKNNRegressor(n_neighbors=5, metric='euclidean')
custom_reg.fit(Xr_train, yr_train)
yr_pred_custom = custom_reg.predict(Xr_test)

mse_custom = mean_squared_error(yr_test, yr_pred_custom)
r2_custom = r2_score(yr_test, yr_pred_custom)
print(f"[Custom KNN Regressor] MSE: {mse_custom:.4f}, R^2: {r2_custom:.4f}")

##############################
# 4.c Улучшение кастомной модели
##############################

# Классификация с улучшенными параметрами
print("\n=== Custom KNN Classifier (Improved) ===")
custom_clf_improved = CustomKNNClassifier(n_neighbors=7, metric='manhattan')
scaler_class = StandardScaler()
Xc_train_scaled = scaler_class.fit_transform(Xc_train)
Xc_test_scaled = scaler_class.transform(Xc_test)

custom_clf_improved.fit(Xc_train_scaled, yc_train)
yc_pred_cust_imp = custom_clf_improved.predict(Xc_test_scaled)

acc_cust_imp = accuracy_score(yc_test, yc_pred_cust_imp)
f1_cust_imp = f1_score(yc_test, yc_pred_cust_imp, average='macro')
print(f"[Custom KNN Classifier Improved] Accuracy: {acc_cust_imp:.4f}, F1-macro: {f1_cust_imp:.4f}")

# Регрессия с улучшенными параметрами
print("\n=== Custom KNN Regressor (Improved) ===")
custom_reg_improved = CustomKNNRegressor(n_neighbors=7, metric='manhattan')
scaler_reg = StandardScaler()
Xr_train_scaled = scaler_reg.fit_transform(Xr_train)
Xr_test_scaled = scaler_reg.transform(Xr_test)

custom_reg_improved.fit(Xr_train_scaled, yr_train)
yr_pred_cust_imp = custom_reg_improved.predict(Xr_test_scaled)

mse_cust_imp = mean_squared_error(yr_test, yr_pred_cust_imp)
r2_cust_imp = r2_score(yr_test, yr_pred_cust_imp)
print(f"[Custom KNN Regressor Improved] MSE: {mse_cust_imp:.4f}, R^2: {r2_cust_imp:.4f}")



[INFO] Breast Cancer dataset loaded.
Features shape: (569, 30), Target shape: (569,)

[INFO] Concrete dataset metadata:
{'uci_id': 165, 'name': 'Concrete Compressive Strength', 'repository_url': 'https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength', 'data_url': 'https://archive.ics.uci.edu/static/public/165/data.csv', 'abstract': 'Concrete is the most important material in civil engineering. The concrete compressive strength is a highly nonlinear function of age and ingredients. ', 'area': 'Physics and Chemistry', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 1030, 'num_features': 8, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Concrete compressive strength'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sun Feb 11 2024', 'dataset_doi': '10.24432/C5PK67', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 383, 'type': 'NATIVE', 'title

Выводы:

1. Бейзлайн модели (KNeighborsClassifier и KNeighborsRegressor из sklearn) показали хорошие показатели качества на обоих датасетах.

2. Улучшение моделей путём гиперпараметрической настройки с использованием GridSearchCV позволило значительно повысить качество моделей как для классификации, так и для регрессии.

3. Собственные реализации KNN (CustomKNNClassifier и CustomKNNRegressor) продемонстрировали основную логику алгоритма K-Nearest Neighbors, сопоставимую с базовыми моделями sklearn.

3. Применение улучшений к собственным моделям оказало разное влияние на классификацию и регрессию: Разрыв в качестве между собственными моделями и моделями из sklearn по-прежнему остаётся значительным, особенно в задаче регрессии.


## Лабораторная работа 2

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.datasets import load_breast_cancer
from ucimlrepo import fetch_ucirepo

# 1. Загрузка данных
# Датасет для классификации (Breast Cancer Wisconsin Dataset)
cancer_data = load_breast_cancer()
X_class = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)
y_class = pd.Series(cancer_data.target)

# Датасет для регрессии (Concrete Compressive Strength)
concrete_data = fetch_ucirepo(id=165)
X_reg = concrete_data.data.features
y_reg = concrete_data.data.targets.squeeze()

# Удаление пропусков и ±∞ значений
X_class = X_class.replace([np.inf, -np.inf], np.nan).dropna()
X_reg = X_reg.replace([np.inf, -np.inf], np.nan).dropna()
y_class = y_class.replace([np.inf, -np.inf], np.nan).dropna()
y_reg = y_reg.replace([np.inf, -np.inf], np.nan).dropna()

# 2. Бейзлайн и оценка качества
# 2.a Разделение на train/test
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# -------------------- ВАЖНО! --------------------
# Для базовой LogisticRegression добавим масштабирование.
# Именно оно обычно решает проблему со сходимостью (или сильно облегчает задачу).
# Для простоты, выполним его прямо здесь. Если вы хотите «чистый» бейзлайн,
# то можно оставить данные как есть, но тогда увеличивать max_iter.
# ------------------------------------------------
scaler_class_baseline = StandardScaler()
Xc_train_baseline_scaled = scaler_class_baseline.fit_transform(Xc_train)
Xc_test_baseline_scaled = scaler_class_baseline.transform(Xc_test)

# 2.b Классификация (Logistic Regression - Бейзлайн)
logreg_baseline = LogisticRegression(max_iter=2000, random_state=42)
logreg_baseline.fit(Xc_train_baseline_scaled, yc_train)
yc_pred_baseline = logreg_baseline.predict(Xc_test_baseline_scaled)

acc = accuracy_score(yc_test, yc_pred_baseline)
prec = precision_score(yc_test, yc_pred_baseline)
rec = recall_score(yc_test, yc_pred_baseline)
f1 = f1_score(yc_test, yc_pred_baseline)

print("\nБейзлайн (Logistic Regression) - Classification:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# 2.c Регрессия (Linear Regression - Бейзлайн)
linreg_baseline = LinearRegression()
linreg_baseline.fit(Xr_train, yr_train)
yr_pred_baseline = linreg_baseline.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred_baseline)
r2 = r2_score(yr_test, yr_pred_baseline)

print("\nБейзлайн (Linear Regression) - Regression:")
print(f"MSE:  {mse:.4f}")
print(f"R^2:  {r2:.4f}")

# 3. Улучшение бейзлайна
# 3.a Обработка данных

# Для классификации - масштабирование данных (уже было показано выше,
# но для «улучшенной» модели можем ещё раз явно получить).
scaler_class = StandardScaler()
Xc_train_scaled = scaler_class.fit_transform(Xc_train)
Xc_test_scaled = scaler_class.transform(Xc_test)

# Для регрессии - добавление полиномиальных признаков + масштабирование
scaler_reg = StandardScaler()
poly = PolynomialFeatures(degree=2, include_bias=False)

Xr_train_scaled = scaler_reg.fit_transform(Xr_train)
Xr_test_scaled = scaler_reg.transform(Xr_test)

# Преобразуем в полиномиальные признаки
Xr_train_poly = poly.fit_transform(Xr_train_scaled)
Xr_test_poly = poly.transform(Xr_test_scaled)

# Допустим, попробуем для «улучшенной» регрессии Ridge
ridge_reg = Ridge(alpha=1.0, random_state=42)
ridge_reg.fit(Xr_train_poly, yr_train)
yr_pred_ridge = ridge_reg.predict(Xr_test_poly)

mse_ridge = mean_squared_error(yr_test, yr_pred_ridge)
r2_ridge = r2_score(yr_test, yr_pred_ridge)

print("\nУлучшенная модель (Ridge + полиномиальные признаки) - Regression:")
print(f"MSE:  {mse_ridge:.4f}")
print(f"R^2:  {r2_ridge:.4f}")

# 3.b Улучшенная модель для классификации (Logistic Regression + GridSearchCV)
pipeline_class = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(random_state=42, max_iter=2000))
])

param_grid_class = {
    "clf__C": [0.01, 0.1, 1, 10, 100],
    "clf__solver": ["lbfgs", "liblinear"]
}

grid_search_class = GridSearchCV(
    estimator=pipeline_class,
    param_grid=param_grid_class,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

# Обучаем на НЕмаcштабированных исходных Xc_train, так как внутри pipeline
# есть свой StandardScaler. (Если уже есть масштабирование, можно было бы
# подавать Xc_train_scaled, но тогда scaler в Pipeline избыточен.)
grid_search_class.fit(Xc_train, yc_train)

best_class_model = grid_search_class.best_estimator_
yc_pred_improved = best_class_model.predict(Xc_test)

acc_improved = accuracy_score(yc_test, yc_pred_improved)
prec_improved = precision_score(yc_test, yc_pred_improved)
rec_improved = recall_score(yc_test, yc_pred_improved)
f1_improved = f1_score(yc_test, yc_pred_improved)

print("\nУлучшенная модель (Logistic Regression + GridSearchCV) - Classification:")
print(f"Accuracy:  {acc_improved:.4f}")
print(f"Precision: {prec_improved:.4f}")
print(f"Recall:    {rec_improved:.4f}")
print(f"F1-score:  {f1_improved:.4f}")

# 4. Собственные реализации моделей
class CustomLinearRegression:
    def __init__(self, lr=0.01, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def fit(self, X, y):
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)

        # Добавляем столбец из 1 для свободного члена
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])

        self.w_ = np.zeros(X.shape[1])

        for i in range(self.n_iter):
            y_pred = X.dot(self.w_)
            error = y_pred - y
            grad = (1 / X.shape[0]) * X.T.dot(error)

            # Проверка на NaN/Inf в градиенте
            if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
                print(f"Iteration {i}: Gradient contains NaN or Inf values, stopping training.")
                break

            self.w_ -= self.lr * grad
        return self

    def predict(self, X):
        X = np.array(X, dtype=float)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        return X.dot(self.w_)


class CustomLogisticRegression:
    def __init__(self, lr=0.0001, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)

        # Добавляем столбец из 1 для свободного члена
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])

        self.w_ = np.zeros(X.shape[1])

        for i in range(self.n_iter):
            z = X.dot(self.w_)
            y_pred = self._sigmoid(z)
            error = y_pred - y
            grad = (1 / X.shape[0]) * X.T.dot(error)

            if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
                print(f"Iteration {i}: Gradient contains NaN or Inf values, stopping training.")
                break

            self.w_ -= self.lr * grad
        return self

    def predict_proba(self, X):
        X = np.array(X, dtype=float)
        ones = np.ones((X.shape[0], 1))
        X = np.hstack([ones, X])
        return self._sigmoid(X.dot(self.w_))

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

# Сравнение с библиотечными методами

# Custom Linear Regression
# Используем те же «улучшенные» данные (полиномиальные признаки + масштабирование)
custom_linreg = CustomLinearRegression(lr=0.01, n_iter=1000)
custom_linreg.fit(Xr_train_poly, yr_train)
yr_pred_custom = custom_linreg.predict(Xr_test_poly)

mse_custom = mean_squared_error(yr_test, yr_pred_custom)
r2_custom = r2_score(yr_test, yr_pred_custom)

print("\nCustom Linear Regression - Regression:")
print(f"MSE:  {mse_custom:.4f}")
print(f"R^2:  {r2_custom:.4f}")

# Custom Logistic Regression
# Возьмём масштабированные данные (без полиномиальных признаков, т.к. это классификация)
custom_logreg = CustomLogisticRegression(lr=0.01, n_iter=2000)
custom_logreg.fit(Xc_train_scaled, yc_train)
yc_pred_custom = custom_logreg.predict(Xc_test_scaled)

acc_custom = accuracy_score(yc_test, yc_pred_custom)
prec_custom = precision_score(yc_test, yc_pred_custom)
rec_custom = recall_score(yc_test, yc_pred_custom)
f1_custom = f1_score(yc_test, yc_pred_custom)

print("\nCustom Logistic Regression - Classification:")
print(f"Accuracy:  {acc_custom:.4f}")
print(f"Precision: {prec_custom:.4f}")
print(f"Recall:    {rec_custom:.4f}")
print(f"F1-score:  {f1_custom:.4f}")





Бейзлайн (Logistic Regression) - Classification:
Accuracy:  0.9825
Precision: 0.9861
Recall:    0.9861
F1-score:  0.9861

Бейзлайн (Linear Regression) - Regression:
MSE:  95.9709
R^2:  0.6276

Улучшенная модель (Ridge + полиномиальные признаки) - Regression:
MSE:  55.1993
R^2:  0.7858

Улучшенная модель (Logistic Regression + GridSearchCV) - Classification:
Accuracy:  0.9737
Precision: 0.9726
Recall:    0.9861
F1-score:  0.9793

Custom Linear Regression - Regression:
MSE:  89.0741
R^2:  0.6543

Custom Logistic Regression - Classification:
Accuracy:  0.9737
Precision: 0.9859
Recall:    0.9722
F1-score:  0.9790


Выводы:

1. Базовые модели (Logistic Regression и Linear Regression из sklearn) продемонстрировали хорошие результаты на соответствующих задачах.

2. Улучшение моделей посредством настройки гиперпараметров и создания полиномиальных признаков значительно повысило качество моделей.

3. Собственные реализации моделей (CustomLinearRegression и CustomLogisticRegression) продемонстрировали конкурентоспособные, но все еще худшие, чем из sklearn результаты, что отмечает важность регуляризации и инженерии признаков для повышения качества модели.


## Лабораторная работа 3

In [10]:
# ==========================================
# Лабораторная работа №3
# (Применение решающего дерева для классификации и регрессии)
# ==========================================

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from ucimlrepo import fetch_ucirepo

# Модели, метрики, инструменты sklearn
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# -----------------------------------------
# 1. Загрузка данных
# -----------------------------------------

# 1.a Данные для задачи классификации - Breast Cancer Wisconsin
breast_cancer_data = load_breast_cancer()
X_bc = breast_cancer_data.data
y_bc = breast_cancer_data.target
feature_names_bc = breast_cancer_data.feature_names

print("=== Breast Cancer Wisconsin Dataset ===")
print("Feature matrix shape:", X_bc.shape)
print("Target vector shape: ", y_bc.shape)
print("Classes (0=malignant, 1=benign)")

# 1.b Данные для задачи регрессии - Concrete Compressive Strength
concrete_data = fetch_ucirepo(id=165)  # Concrete Compressive Strength Dataset
X_conc = concrete_data.data.features
y_conc = concrete_data.data.targets  # Это Series с целевым значением прочности
feature_names_conc = X_conc.columns

print("\n=== Concrete Compressive Strength Dataset ===")
print("Feature matrix shape:", X_conc.shape)
print("Target vector shape: ", y_conc.shape)

# -----------------------------------------
# 2. Создание бейзлайна и оценка качества
# -----------------------------------------

# 2.a ОБУЧЕНИЕ (Classification)

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_bc, y_bc, test_size=0.2, random_state=42, stratify=y_bc
)

# Бейзлайновая модель: DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(Xc_train, yc_train)
yc_pred = dt_clf.predict(Xc_test)

# 2.b ОЦЕНКА КАЧЕСТВА (Classification)
acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("\n=== Бейзлайн DecisionTree (Classification) ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# 2.a ОБУЧЕНИЕ (Regression)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_conc, y_conc, test_size=0.2, random_state=42
)

# Бейзлайновая модель: DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(Xr_train, yr_train)
yr_pred = dt_reg.predict(Xr_test)

# 2.b ОЦЕНКА КАЧЕСТВА (Regression)
mse = mean_squared_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\n=== Бейзлайн DecisionTree (Regression) ===")
print(f"MSE:  {mse:.4f}")
print(f"R^2:  {r2:.4f}")

# -----------------------------------------
# 3. Улучшение бейзлайна
# -----------------------------------------

# 3.a Формулировка гипотез улучшения:
#    1) Подбор гиперпараметров решающего дерева (max_depth, min_samples_leaf, min_samples_split, ...)
#    2) Масштабирование (для некоторых моделей), кодирование категориальных признаков (если были)
#    3) Добавление новых признаков (если имеет смысл; в данном примере все признаки уже числовые)
#    4) Удаление выбросов (при необходимости)

# В нашем случае (Breast Cancer) все признаки числовые и уже относительно
# отмасштабированы. Для примера - мы просто подберём гиперпараметры.

print("\n=== Улучшение бейзлайна: Классификация (Breast Cancer) ===")

param_grid_clf = {
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_clf = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_clf,
    cv=5,
    scoring='f1',
    n_jobs=-1
)
grid_clf.fit(Xc_train, yc_train)

best_clf = grid_clf.best_estimator_
yc_pred_best = best_clf.predict(Xc_test)

acc_best = accuracy_score(yc_test, yc_pred_best)
prec_best = precision_score(yc_test, yc_pred_best)
rec_best = recall_score(yc_test, yc_pred_best)
f1_best = f1_score(yc_test, yc_pred_best)

print("Лучшие параметры:", grid_clf.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

print("\n=== Улучшение бейзлайна: Регрессия (Concrete Strength) ===")

# Для демонстрации: попробуем добавить искусственный признак ratio_water_cement,
# если он ещё не существует, — отношение воды к цементу,
# полагая, что это может быть важным фактором в задаче.

if 'Water' in X_conc.columns and 'Cement' in X_conc.columns:
    X_conc['ratio_water_cement'] = X_conc['Water'] / (X_conc['Cement'] + 1e-5)

# Разбиваем заново, теперь уже с новым признаком
Xr_train2, Xr_test2, yr_train2, yr_test2 = train_test_split(
    X_conc, y_conc, test_size=0.2, random_state=42
)

# Создадим пайплайн для подбора гиперпараметров
# (при желании можно добавить StandardScaler, но DecisionTree не слишком
#  чувствительна к масштабу)
pipe_reg = Pipeline([
    ("dt_reg", DecisionTreeRegressor(random_state=42))
])

param_grid_reg = {
    "dt_reg__max_depth": [3, 5, 7, None],
    "dt_reg__min_samples_leaf": [1, 2, 5],
    "dt_reg__min_samples_split": [2, 5, 10],
    "dt_reg__criterion": ["squared_error", "absolute_error"]
}

grid_reg = GridSearchCV(
    pipe_reg,
    param_grid_reg,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_reg.fit(Xr_train2, yr_train2)
best_reg = grid_reg.best_estimator_
yr_pred_best = best_reg.predict(Xr_test2)

mse_best = mean_squared_error(yr_test2, yr_pred_best)
r2_best = r2_score(yr_test2, yr_pred_best)

print("Лучшие параметры:", grid_reg.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"R^2:  {r2_best:.4f}")

# -----------------------------------------
# 4. Имплементация "Custom Decision Tree"
#    (здесь - упрощённый пример)
# -----------------------------------------
from collections import Counter

class CustomDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.root = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth):
        # 1) критерии остановки
        if len(set(y)) == 1:
            return {'type':'leaf','class':y[0]}
        if (self.max_depth is not None) and (depth >= self.max_depth):
            # Мажоритарный класс
            return {'type':'leaf','class': Counter(y).most_common(1)[0][0]}
        if len(X) < self.min_samples_split:
            return {'type':'leaf','class': Counter(y).most_common(1)[0][0]}

        # 2) Упрощённый выбор лучшего признака: первый признак и медиана
        best_feat = 0
        best_thresh = np.median(X[:,0])

        left_idx = X[:,best_feat] <= best_thresh
        right_idx = ~left_idx

        node = {
            'type': 'node',
            'feature': best_feat,
            'thresh': best_thresh,
            'left':  self._build_tree(X[left_idx],  y[left_idx],  depth+1),
            'right': self._build_tree(X[right_idx], y[right_idx], depth+1)
        }
        return node

    def predict(self, X):
        X = np.array(X)
        preds = []
        for x in X:
            preds.append(self._traverse(self.root, x))
        return np.array(preds)

    def _traverse(self, node, x):
        if node['type'] == 'leaf':
            return node['class']
        else:
            if x[node['feature']] <= node['thresh']:
                return self._traverse(node['left'], x)
            else:
                return self._traverse(node['right'], x)

class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.root = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth):
        # Критерии остановки
        if (self.max_depth is not None) and (depth >= self.max_depth):
            return {'type':'leaf', 'value': y.mean()}
        if len(X) < self.min_samples_split:
            return {'type':'leaf', 'value': y.mean()}
        if np.allclose(y, y[0]):
            return {'type':'leaf', 'value': y[0]}

        # Упрощённый вариант сплита - первый признак, медиана
        best_feat = 0
        best_thresh = np.median(X[:,0])

        left_idx = X[:,best_feat] <= best_thresh
        right_idx = ~left_idx

        node = {
            'type': 'node',
            'feature': best_feat,
            'thresh': best_thresh,
            'left':  self._build_tree(X[left_idx], y[left_idx], depth+1),
            'right': self._build_tree(X[right_idx], y[right_idx], depth+1)
        }
        return node

    def predict(self, X):
        X = np.array(X)
        preds = []
        for x in X:
            preds.append(self._traverse(self.root, x))
        return np.array(preds)

    def _traverse(self, node, x):
        if node['type'] == 'leaf':
            return node['value']
        else:
            if x[node['feature']] <= node['thresh']:
                return self._traverse(node['left'], x)
            else:
                return self._traverse(node['right'], x)

# -----------------------------------------
# 4.a-b Обучение кастомных моделей и оценка
# -----------------------------------------

# Классификация на Breast Cancer
cust_clf = CustomDecisionTreeClassifier(max_depth=3, min_samples_split=10)
cust_clf.fit(Xc_train, yc_train)
yc_pred_custom = cust_clf.predict(Xc_test)

acc_cust = accuracy_score(yc_test, yc_pred_custom)
f1_cust = f1_score(yc_test, yc_pred_custom)

print("\n=== Custom Decision Tree (Classification) ===")
print(f"Accuracy: {acc_cust:.4f}")
print(f"F1-score: {f1_cust:.4f}")

# Регрессия на Concrete Strength
cust_reg = CustomDecisionTreeRegressor(max_depth=3, min_samples_split=10)
cust_reg.fit(Xr_train, yr_train)
yr_pred_custom = cust_reg.predict(Xr_test)

mse_cust = mean_squared_error(yr_test, yr_pred_custom)
r2_cust = r2_score(yr_test, yr_pred_custom)

print("\n=== Custom Decision Tree (Regression) ===")
print(f"MSE: {mse_cust:.4f}")
print(f"R^2: {r2_cust:.4f}")

# -----------------------------------------
# 4.f-g Применение улучшений для кастомной модели
# -----------------------------------------

# Пример небольшого улучшения: добавляем искусственный признак
# (Breast Cancer: возьмём два первых признака и сделаем их сумму)

Xc_train_enh = np.column_stack([Xc_train, Xc_train[:,0] + Xc_train[:,1]])
Xc_test_enh = np.column_stack([Xc_test,  Xc_test[:,0]  + Xc_test[:,1]])

cust_clf_improved = CustomDecisionTreeClassifier(max_depth=5, min_samples_split=5)
cust_clf_improved.fit(Xc_train_enh, yc_train)
yc_pred_cust_improved = cust_clf_improved.predict(Xc_test_enh)

acc_cust_improved = accuracy_score(yc_test, yc_pred_cust_improved)
f1_cust_improved = f1_score(yc_test, yc_pred_cust_improved)

print("\n=== Custom Decision Tree Improved (Classification) ===")
print(f"Accuracy: {acc_cust_improved:.4f}")
print(f"F1-score: {f1_cust_improved:.4f}")

# Аналогично для регрессии (Concrete Strength):
# Добавим признак = сумма Cement и Water
Xr_train_enh = Xr_train.copy()
Xr_test_enh = Xr_test.copy()

if 'Cement' in Xr_train.columns and 'Water' in Xr_train.columns:
    Xr_train_enh['cement_water_sum'] = Xr_train['Cement'] + Xr_train['Water']
    Xr_test_enh['cement_water_sum']  = Xr_test['Cement'] + Xr_test['Water']

cust_reg_improved = CustomDecisionTreeRegressor(max_depth=5, min_samples_split=5)
cust_reg_improved.fit(Xr_train_enh, yr_train)
yr_pred_cust_improved = cust_reg_improved.predict(Xr_test_enh)

mse_cust_improved = mean_squared_error(yr_test, yr_pred_cust_improved)
r2_cust_improved = r2_score(yr_test, yr_pred_cust_improved)

print("\n=== Custom Decision Tree Improved (Regression) ===")
print(f"MSE:  {mse_cust_improved:.4f}")
print(f"R^2:  {r2_cust_improved:.4f}")


=== Breast Cancer Wisconsin Dataset ===
Feature matrix shape: (569, 30)
Target vector shape:  (569,)
Classes (0=malignant, 1=benign)

=== Concrete Compressive Strength Dataset ===
Feature matrix shape: (1030, 8)
Target vector shape:  (1030, 1)

=== Бейзлайн DecisionTree (Classification) ===
Accuracy:  0.9123
Precision: 0.9559
Recall:    0.9028
F1-score:  0.9286

=== Бейзлайн DecisionTree (Regression) ===
MSE:  42.0228
R^2:  0.8369

=== Улучшение бейзлайна: Классификация (Breast Cancer) ===
Лучшие параметры: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}
Accuracy:  0.9211
Precision: 0.9565
Recall:    0.9167
F1-score:  0.9362

=== Улучшение бейзлайна: Регрессия (Concrete Strength) ===
Лучшие параметры: {'dt_reg__criterion': 'squared_error', 'dt_reg__max_depth': None, 'dt_reg__min_samples_leaf': 1, 'dt_reg__min_samples_split': 2}
MSE:  48.4651
R^2:  0.8119

=== Custom Decision Tree (Classification) ===
Accuracy: 0.8596
F1-score: 0.8857

=== Custom Dec

Выводы:
1. Бейзлайновые модели sklearn (DecisionTreeClassifier/Regressor) обычно дают
   более качественные предсказания, чем простая кастомная реализация,
   потому что в sklearn применены более совершенные алгоритмы поиска оптимального сплита.
2. Подбор гиперпараметров и добавление новых признаков улучшают показатели
   как в sklearn-модели, так и в кастомной.
3. В реальных задачах рекомендуется использовать хорошо отлаженные библиотеки, а кастомные реализации полезны
   только в образовательных/исследовательских целях, чтобы глубже понять логику работы алгоритма.

## Лабораторная работа 4

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score)
from sklearn.model_selection import train_test_split, GridSearchCV
from collections import Counter

from ucimlrepo import fetch_ucirepo

###############################################################################
# 1) Загрузка данных
###############################################################################

# --- (A) Датасет для классификации: Breast Cancer Wisconsin ---
breast_data = load_breast_cancer()
X_bc = pd.DataFrame(breast_data.data, columns=breast_data.feature_names)
y_bc = pd.Series(breast_data.target, name='target')  # превращаем в Series

print("Breast Cancer data shape:", X_bc.shape)

# --- (B) Датасет для регрессии: Concrete Compressive Strength ---
concrete_dataset = fetch_ucirepo(id=165)  # ID=165 - Concrete Compressive Strength
X_cc = concrete_dataset.data.features
y_cc = concrete_dataset.data.targets

print("Concrete data shape:", X_cc.shape)


###############################################################################
# 2) Первичная обработка / Бейзлайн
###############################################################################

# --- 2.A Классификация (Breast Cancer) ---

# Пример: удаляем строки, где mean texture > 40 (условные "выбросы")
X_bc_clean = X_bc[X_bc['mean texture'] < 40].copy()
y_bc_clean = y_bc.loc[X_bc_clean.index].squeeze()

# Создадим новый признак, например: mean radius * mean smoothness
X_bc_clean['radius_smoothness'] = X_bc_clean['mean radius'] * X_bc_clean['mean smoothness']

# Разделение на train/test
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_bc_clean, y_bc_clean, test_size=0.2, random_state=0, stratify=y_bc_clean
)

# Бейзлайн: RandomForestClassifier без подбора гиперпараметров
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(Xc_train, y_bc_clean.loc[Xc_train.index].to_numpy())  # Используем to_numpy()
yc_pred = rf_clf.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("\nБейзлайн RandomForest (Classification) на Breast Cancer:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")


# --- 2.B Регрессия (Concrete) ---

# Пример: удаляем строки, где Cement > 550 (условные "выбросы")
X_cc_clean = X_cc[X_cc['Cement'] <= 550].copy()
y_cc_clean = y_cc.loc[X_cc_clean.index].squeeze()

# Создадим новый признак: Water * Age
X_cc_clean['Water*Age'] = X_cc_clean['Water'] * X_cc_clean['Age']

# Разделение на train/test
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_cc_clean, y_cc_clean, test_size=0.2, random_state=0
)

# Бейзлайн: RandomForestRegressor без подбора гиперпараметров
rf_reg = RandomForestRegressor(random_state=0)
rf_reg.fit(Xr_train, y_cc_clean.loc[Xr_train.index].to_numpy())  # Используем to_numpy()
yr_pred = rf_reg.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\nБейзлайн RandomForest (Regression) на Concrete Compressive Strength:")
print(f"MSE:  {mse:.4f}")
print(f"R^2:  {r2:.4f}")


###############################################################################
# 3) Улучшение бейзлайна (подбор гиперпараметров)
###############################################################################

# --- 3.A Classification (Breast Cancer) ---
param_grid_clf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}

rf_clf_cv = GridSearchCV(
    RandomForestClassifier(random_state=0),
    param_grid_clf,
    cv=5,
    scoring='f1',
    n_jobs=-1
)
rf_clf_cv.fit(Xc_train, y_bc_clean.loc[Xc_train.index].to_numpy())  # Используем to_numpy()
best_clf = rf_clf_cv.best_estimator_
yc_pred_best = best_clf.predict(Xc_test)

acc_best = accuracy_score(yc_test, yc_pred_best)
prec_best = precision_score(yc_test, yc_pred_best)
rec_best = recall_score(yc_test, yc_pred_best)
f1_best = f1_score(yc_test, yc_pred_best)

print("\nУлучшенный RandomForest (Classification):")
print("Лучшие параметры:", rf_clf_cv.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")


# --- 3.B Regression (Concrete) ---
param_grid_reg = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, None],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}

rf_reg_cv = GridSearchCV(
    RandomForestRegressor(random_state=0),
    param_grid_reg,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
rf_reg_cv.fit(Xr_train, y_cc_clean.loc[Xr_train.index].to_numpy())  # Используем to_numpy()
best_reg = rf_reg_cv.best_estimator_
yr_pred_best = best_reg.predict(Xr_test)

mse_best = mean_squared_error(yr_test, yr_pred_best)
r2_best = r2_score(yr_test, yr_pred_best)

print("\nУлучшенный RandomForest (Regression):")
print("Лучшие параметры:", rf_reg_cv.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"R^2:  {r2_best:.4f}")


###############################################################################
# 4) Имплементация собственного (упрощённого) RandomForest
###############################################################################

class SimpleDecisionTreeClassifier:
    """
    Очень упрощённое дерево для классификации.
    """
    def __init__(self, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.root = None
        if self.random_state is not None:
            self.random_state_ = np.random.RandomState(self.random_state)
        else:
            self.random_state_ = np.random.RandomState()

    def _get_features_subset(self, n_features):
        """Выбираем случайное подмножество признаков согласно max_features"""
        if self.max_features is None:
            return np.arange(n_features)
        elif self.max_features == 'sqrt':
            k = max(1, int(np.sqrt(n_features)))
            return self.random_state_.choice(n_features, k, replace=False)
        elif isinstance(self.max_features, int):
            k = min(self.max_features, n_features)
            return self.random_state_.choice(n_features, k, replace=False)
        else:
            raise ValueError("max_features должно быть 'sqrt', int или None")

    def fit(self, X, y, depth=0):
        X = np.array(X)
        y = np.array(y)

        # Проверка на пустые
        if len(X) == 0 or len(y) == 0:
            self.root = ('leaf', 0)
            return self

        # Критерии остановки
        if len(np.unique(y)) == 1 or len(X) < self.min_samples_split:
            self.root = ('leaf', Counter(y).most_common(1)[0][0])
            return self
        if self.max_depth is not None and depth >= self.max_depth:
            self.root = ('leaf', Counter(y).most_common(1)[0][0])
            return self

        n_features = X.shape[1]
        features_subset = self._get_features_subset(n_features)

        best_feat = None
        best_thresh = None
        best_balance = len(X)

        for feat in features_subset:
            thresh_candidate = np.median(X[:, feat])
            left_idx = (X[:, feat] <= thresh_candidate)
            right_idx = ~left_idx

            # если одна из сторон пуста - пропускаем
            if left_idx.sum() == 0 or right_idx.sum() == 0:
                continue

            # простой критерий "баланс"
            balance = abs(left_idx.sum() - right_idx.sum())
            if balance < best_balance:
                best_balance = balance
                best_feat = feat
                best_thresh = thresh_candidate

        if best_feat is None:
            # если не нашли признака, делаем лист
            self.root = ('leaf', Counter(y).most_common(1)[0][0])
            return self

        left_idx = (X[:, best_feat] <= best_thresh)
        right_idx = ~left_idx

        # Генерируем новые случайные состояния для поддеревьев
        left_random_state = self.random_state_.randint(0, 10000)
        right_random_state = self.random_state_.randint(0, 10000)

        left_tree = SimpleDecisionTreeClassifier(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_features=self.max_features,
            random_state=left_random_state
        ).fit(X[left_idx], y[left_idx], depth+1)

        right_tree = SimpleDecisionTreeClassifier(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_features=self.max_features,
            random_state=right_random_state
        ).fit(X[right_idx], y[right_idx], depth+1)

        self.root = ('node', best_feat, best_thresh, left_tree, right_tree)
        return self

    def predict_one(self, x):
        node = self.root
        while True:
            if node[0] == 'leaf':
                return node[1]
            else:
                _, feat, thresh, left_subtree, right_subtree = node
                if x[feat] <= thresh:
                    node = left_subtree.root
                else:
                    node = right_subtree.root

    def predict(self, X):
        X = np.array(X)
        return np.array([self.predict_one(x) for x in X])


class SimpleDecisionTreeRegressor:
    """
    Упрощённое дерево для регрессии.
    """
    def __init__(self, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.root = None
        if self.random_state is not None:
            self.random_state_ = np.random.RandomState(self.random_state)
        else:
            self.random_state_ = np.random.RandomState()

    def _get_features_subset(self, n_features):
        """Выбираем случайное подмножество признаков согласно max_features"""
        if self.max_features is None:
            return np.arange(n_features)
        elif self.max_features == 'sqrt':
            k = max(1, int(np.sqrt(n_features)))
            return self.random_state_.choice(n_features, k, replace=False)
        elif isinstance(self.max_features, int):
            k = min(self.max_features, n_features)
            return self.random_state_.choice(n_features, k, replace=False)
        else:
            raise ValueError("max_features должно быть 'sqrt', int или None")

    def fit(self, X, y, depth=0):
        X = np.array(X)
        y = np.array(y, dtype=float)

        # Проверка на пустые
        if len(X) == 0 or len(y) == 0:
            self.root = ('leaf', 0.0)
            return self

        # Критерии остановки
        if len(X) < self.min_samples_split:
            self.root = ('leaf', np.mean(y))
            return self
        if self.max_depth is not None and depth >= self.max_depth:
            self.root = ('leaf', np.mean(y))
            return self
        if np.allclose(y, y[0]):
            self.root = ('leaf', y[0])
            return self

        n_features = X.shape[1]
        features_subset = self._get_features_subset(n_features)

        best_feat = None
        best_thresh = None
        best_balance = len(X)

        for feat in features_subset:
            thresh_candidate = np.median(X[:, feat])
            left_idx = (X[:, feat] <= thresh_candidate)
            right_idx = ~left_idx

            # если одна из сторон пуста - пропускаем
            if left_idx.sum() == 0 or right_idx.sum() == 0:
                continue

            # простой критерий "баланс"
            balance = abs(left_idx.sum() - right_idx.sum())
            if balance < best_balance:
                best_balance = balance
                best_feat = feat
                best_thresh = thresh_candidate

        if best_feat is None:
            # если не нашли признака, делаем лист
            self.root = ('leaf', np.mean(y))
            return self

        left_idx = (X[:, best_feat] <= best_thresh)
        right_idx = ~left_idx

        # Генерируем новые случайные состояния для поддеревьев
        left_random_state = self.random_state_.randint(0, 10000)
        right_random_state = self.random_state_.randint(0, 10000)

        left_tree = SimpleDecisionTreeRegressor(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_features=self.max_features,
            random_state=left_random_state
        ).fit(X[left_idx], y[left_idx], depth+1)

        right_tree = SimpleDecisionTreeRegressor(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            max_features=self.max_features,
            random_state=right_random_state
        ).fit(X[right_idx], y[right_idx], depth+1)

        self.root = ('node', best_feat, best_thresh, left_tree, right_tree)
        return self

    def predict_one(self, x):
        node = self.root
        while True:
            if node[0] == 'leaf':
                return node[1]
            else:
                _, feat, thresh, left_subtree, right_subtree = node
                if x[feat] <= thresh:
                    node = left_subtree.root
                else:
                    node = right_subtree.root

    def predict(self, X):
        X = np.array(X)
        return np.array([self.predict_one(x) for x in X])


class SimpleRandomForestClassifier:
    """
    Упрощённая реализация RandomForest для классификации.
    """
    def __init__(self, n_estimators=50, max_depth=None, min_samples_split=2,
                 max_features='sqrt', random_state=0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples = X.shape[0]

        self.trees = []
        rng = np.random.RandomState(self.random_state)

        for i in range(self.n_estimators):
            # Bootstrap
            indices = rng.choice(n_samples, n_samples, replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            # Создаём дерево с уникальным random_state для разнообразия
            tree_random_state = rng.randint(0, 10000)
            tree = SimpleDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
                random_state=tree_random_state
            )
            tree.fit(X_boot, y_boot)
            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.array(X)
        # Сбор предсказаний от всех деревьев
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Транспонируем для удобства
        predictions = predictions.T  # shape: (n_samples, n_estimators)

        final_preds = []
        for i in range(predictions.shape[0]):
            column = predictions[i]
            most_common = Counter(column).most_common(1)[0][0]
            final_preds.append(most_common)
        return np.array(final_preds)


class SimpleRandomForestRegressor:
    """
    Упрощённая реализация RandomForest для регрессии.
    """
    def __init__(self, n_estimators=50, max_depth=None, min_samples_split=2,
                 max_features='sqrt', random_state=0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples = X.shape[0]

        self.trees = []
        rng = np.random.RandomState(self.random_state)

        for i in range(self.n_estimators):
            # Bootstrap
            indices = rng.choice(n_samples, n_samples, replace=True)
            X_boot = X[indices]
            y_boot = y[indices]

            # Создаём дерево с уникальным random_state для разнообразия
            tree_random_state = rng.randint(0, 10000)
            tree = SimpleDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
                random_state=tree_random_state
            )
            tree.fit(X_boot, y_boot)
            self.trees.append(tree)

        return self

    def predict(self, X):
        X = np.array(X)
        # Сбор предсказаний от всех деревьев
        all_preds = np.array([tree.predict(X) for tree in self.trees])  # shape: (n_estimators, n_samples)
        # Среднее по деревьям
        return np.mean(all_preds, axis=0)


###############################################################################
# Обучение и Оценка Собственной Реализации RandomForest (Классификация)
###############################################################################


print("Обучение и оценка собственной реализации RandomForest (Classification)")


# Инициализация собственной реализации RandomForestClassifier
cust_rf_clf = SimpleRandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    min_samples_split=2,
    max_features='sqrt',
    random_state=0
)

# Обучение на тех же данных
cust_rf_clf.fit(Xc_train, yc_train)

# Предсказание на тестовой выборке
yc_pred_custom = cust_rf_clf.predict(Xc_test)

# Вычисление метрик
acc_cust = accuracy_score(yc_test, yc_pred_custom)
prec_cust = precision_score(yc_test, yc_pred_custom)
rec_cust = recall_score(yc_test, yc_pred_custom)
f1_cust = f1_score(yc_test, yc_pred_custom)

# Вывод результатов
print("\n[Custom RandomForestClassifier] (Breast Cancer)")
print(f"Accuracy:  {acc_cust:.4f}")
print(f"Precision: {prec_cust:.4f}")
print(f"Recall:    {rec_cust:.4f}")
print(f"F1-score:  {f1_cust:.4f}")


###############################################################################
# Обучение и Оценка Собственной Реализации RandomForest (Регрессия)
###############################################################################

print("Обучение и оценка собственной реализации RandomForest (Regression)")


# Инициализация собственной реализации RandomForestRegressor
cust_rf_reg = SimpleRandomForestRegressor(
    n_estimators=50,
    max_depth=5,
    min_samples_split=2,
    max_features='sqrt',
    random_state=0
)

# Обучение на тех же данных
cust_rf_reg.fit(Xr_train, yr_train)

# Предсказание на тестовой выборке
yr_pred_custom = cust_rf_reg.predict(Xr_test)

# Вычисление метрик
mse_cust = mean_squared_error(yr_test, yr_pred_custom)
r2_cust = r2_score(yr_test, yr_pred_custom)

# Вывод результатов
print("\n[Custom RandomForestRegressor] (Concrete)")
print(f"MSE:  {mse_cust:.4f}")
print(f"R^2:  {r2_cust:.4f}")


Breast Cancer data shape: (569, 30)
Concrete data shape: (1030, 8)

Бейзлайн RandomForest (Classification) на Breast Cancer:
Accuracy:  0.9474
Precision: 0.9583
Recall:    0.9583
F1-score:  0.9583

Бейзлайн RandomForest (Regression) на Concrete Compressive Strength:
MSE:  20.9711
R^2:  0.9204

Улучшенный RandomForest (Classification):
Лучшие параметры: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 50}
Accuracy:  0.9474
Precision: 0.9583
Recall:    0.9583
F1-score:  0.9583

Улучшенный RandomForest (Regression):
Лучшие параметры: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 100}
MSE:  21.0961
R^2:  0.9199
Обучение и оценка собственной реализации RandomForest (Classification)

[Custom RandomForestClassifier] (Breast Cancer)
Accuracy:  0.9474
Precision: 0.9459
Recall:    0.9722
F1-score:  0.9589
Обучение и оценка собственной реализации RandomForest (Regression)

[Custom RandomForestRegressor] (Concrete)
MSE:  98.2264
R

Выводы:

1. Базовые модели (RandomForestClassifier и RandomForestRegressor из sklearn) показали высокую эффективность.

2. Оптимизация гиперпараметров с помощью GridSearchCV не привела к существенному улучшению:
Классификация: Улучшенная модель показала аналогичные результаты базовой модели, что свидетельствует о близости гиперпараметров к оптимальным.
Регрессия: Небольшое ухудшение показателей после настройки, что может быть связано с переобучением или неидеальным подбором параметров.

3. Собственные реализации моделей продемонстрировали разную эффективность:
Классификация: Custom RandomForestClassifier показал результаты, сопоставимые с моделями sklearn, подтверждая корректную реализацию алгоритма.
Регрессия: Custom RandomForestRegressor значительно уступил библиотечным моделям по MSE и R², указывая на необходимость доработки алгоритма.

4. Готовые реализации RandomForest из sklearn обеспечивают высокую производительность и надежность для практических задач. Разработка собственных моделей полезна для глубокого понимания алгоритмов, однако требует дальнейшей оптимизации для достижения конкурентоспособных результатов, особенно в задачах регрессии.

## Лабораторная работа 5

In [11]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score)
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

from ucimlrepo import fetch_ucirepo

# ----------------------------
# 1. Загрузка данных
# ----------------------------

## 1.a Классификация: Breast Cancer Wisconsin Dataset
breast_data = load_breast_cancer()
X_class = breast_data.data
y_class = breast_data.target

print("Breast Cancer Dataset shape:", X_class.shape)
print("Классы меток:", np.unique(y_class))

# Разделение данных на обучающую и тестовую выборки
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

## 1.b Регрессия: Concrete Compressive Strength Dataset
concrete_compressive_strength = fetch_ucirepo(id=165)

# Проверим структуру объекта, чтобы корректно извлечь имена переменных
print("\nСтруктура concrete_compressive_strength.variables:")
print(concrete_compressive_strength.variables)

# Предполагая, что concrete_compressive_strength.variables - это DataFrame
# с столбцом 'name' и 'role', извлечём имена только признаков (role == 'Feature')
if isinstance(concrete_compressive_strength.variables, pd.DataFrame):
    # Отбираем только признаки
    features_df = concrete_compressive_strength.variables[concrete_compressive_strength.variables['role'] == 'Feature']
    column_names = features_df['name'].tolist()
else:
    raise ValueError("Непредвиденная структура concrete_compressive_strength.variables")

# Преобразование данных в pandas DataFrame для удобства
X_reg = pd.DataFrame(concrete_compressive_strength.data.features, columns=column_names)

# Исправление: преобразование целевой переменной в одномерный массив
y_reg = concrete_compressive_strength.data.targets.squeeze()

print("\nConcrete Compressive Strength Dataset shape:", X_reg.shape)
print("Первые 5 строк признаков:\n", X_reg.head())

# Разделение данных на обучающую и тестовую выборки
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# ----------------------------
# 2. Создание бейзлайна и оценка качества
# ----------------------------

## 2.a Обучение моделей из sklearn

### Бейзлайн (Классификация)
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(Xc_train, yc_train)
yc_pred = gb_clf.predict(Xc_test)

# Оценка качества модели классификации
acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("\nБейзлайн GradientBoosting (Классификация - Breast Cancer):")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

### Бейзлайн (Регрессия)
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(Xr_train, yr_train)
yr_pred = gb_reg.predict(Xr_test)

# Оценка качества модели регрессии
mse = mean_squared_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\nБейзлайн GradientBoosting (Регрессия - Concrete Strength):")
print(f"MSE:  {mse:.4f}")
print(f"R^2:  {r2:.4f}")

# ----------------------------
# 3. Улучшение бейзлайна
# ----------------------------

## 3.a Формулировка гипотез
# Для классификации: отбор наиболее информативных признаков
# Для регрессии: создание новых признаков путем взаимодействия существующих

## 3.b Проверка гипотез

### Улучшение данных для классификации: отбор признаков с высокой важностью
# Получим важности признаков из бейзлайна
feature_importances = gb_clf.feature_importances_
features = breast_data.feature_names
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
importance_df = importance_df.sort_values(by='importance', ascending=False)

print("\nВажности признаков (Классификация):")
print(importance_df.head())

# Выберем топ-5 признаков
top_features = importance_df['feature'].iloc[:5].tolist()
print("\nОтобранные признаки для классификации:", top_features)

# Извлечём только выбранные признаки для обучающей и тестовой выборок
Xc_train_improved = pd.DataFrame(Xc_train, columns=features)[top_features]
Xc_test_improved  = pd.DataFrame(Xc_test, columns=features)[top_features]

### Улучшение данных для регрессии: создание нового признака
X_reg_improved = X_reg.copy()

# Создадим новый признак: взаимодействие цемента и возраста
cement_col = 'Cement'
age_col = 'Age'

if cement_col in X_reg_improved.columns and age_col in X_reg_improved.columns:
    X_reg_improved['Cement_Age'] = X_reg_improved[cement_col] * X_reg_improved[age_col]
    print(f"\nСоздан новый признак 'Cement_Age' как произведение '{cement_col}' и '{age_col}'.")
else:
    raise KeyError(f"Один из столбцов '{cement_col}' или '{age_col}' отсутствует в данных.")

# Обновим обучающую и тестовую выборки
Xr_train_improved, Xr_test_improved, yr_train_improved, yr_test_improved = train_test_split(
    X_reg_improved, y_reg, test_size=0.2, random_state=42
)

## 3.c Обновлённый бейзлайн

### Улучшенный бейзлайн (Классификация)
param_grid_clf = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
}

gb_clf_cv = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid_clf,
    cv=5,
    scoring='f1',
    n_jobs=-1
)
gb_clf_cv.fit(Xc_train_improved, yc_train)
best_clf = gb_clf_cv.best_estimator_

yc_pred_improved = best_clf.predict(Xc_test_improved)

acc_best = accuracy_score(yc_test, yc_pred_improved)
prec_best = precision_score(yc_test, yc_pred_improved)
rec_best = recall_score(yc_test, yc_pred_improved)
f1_best = f1_score(yc_test, yc_pred_improved)

print("\nУлучшенный бейзлайн GradientBoosting (Классификация):")
print("Лучшие параметры:", gb_clf_cv.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

### Улучшенный бейзлайн (Регрессия)
param_grid_reg = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
}

gb_reg_cv = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid_reg,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
gb_reg_cv.fit(Xr_train_improved, yr_train_improved)
best_reg = gb_reg_cv.best_estimator_

yr_pred_improved = best_reg.predict(Xr_test_improved)

mse_best = mean_squared_error(yr_test_improved, yr_pred_improved)
r2_best = r2_score(yr_test_improved, yr_pred_improved)

print("\nУлучшенный бейзлайн GradientBoosting (Регрессия):")
print("Лучшие параметры:", gb_reg_cv.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"R^2:  {r2_best:.4f}")

# ----------------------------
# 4. Имплементация собственного алгоритма градиентного бустинга
# ----------------------------

## 4.a Реализация простого регрессора-пеньки
class SimpleDecisionStumpRegressor:
    """
    Простейшая модель дерева глубины 1:
    - Один признак
    - Один порог (медиана)
    - Предсказание = среднее слева и справа
    """
    def __init__(self):
        self.feat = None
        self.thresh = None
        self.left_value = None
        self.right_value = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        # Для упрощения выбираем признак с наибольшей дисперсией
        variances = X.var(axis=0)
        self.feat = np.argmax(variances)
        self.thresh = np.median(X[:, self.feat])

        left_idx = X[:, self.feat] <= self.thresh
        right_idx = ~left_idx

        self.left_value = y[left_idx].mean() if np.any(left_idx) else 0
        self.right_value = y[right_idx].mean() if np.any(right_idx) else 0
        return self

    def predict(self, X):
        X = np.array(X)
        pred = np.zeros(X.shape[0])
        left_idx = X[:, self.feat] <= self.thresh
        right_idx = ~left_idx
        pred[left_idx] = self.left_value
        pred[right_idx] = self.right_value
        return pred

## 4.b Реализация простого градиентного бустинга для регрессии
class SimpleGBRegressor:
    """
    Упрощённый градиентный бустинг для регрессии:
    - f0 = среднее y
    - f_m(x) = f_{m-1}(x) + lr * h_m(x)
    """
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.f0 = 0

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y, dtype=float)

        self.f0 = np.mean(y)
        f_current = np.ones_like(y) * self.f0

        self.models = []
        for _ in range(self.n_estimators):
            residuals = y - f_current
            stump = SimpleDecisionStumpRegressor()
            stump.fit(X, residuals)
            self.models.append(stump)
            f_current += self.learning_rate * stump.predict(X)
        return self

    def predict(self, X):
        X = np.array(X)
        preds = np.ones(X.shape[0]) * self.f0
        for stump in self.models:
            preds += self.learning_rate * stump.predict(X)
        return preds

## 4.c Реализация простого классификатора градиентного бустинга
class SimpleGBClassifier:
    """
    Упрощённый градиентный бустинг для классификации 0/1:
    - Оптимизируем MSE между предсказанием и метками
    - Применяем сигмоиду для получения вероятности
    """
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.f0 = 0

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y, dtype=float)

        self.f0 = np.mean(y)
        f_current = np.ones_like(y) * self.f0

        self.models = []
        for _ in range(self.n_estimators):
            residuals = y - f_current
            stump = SimpleDecisionStumpRegressor()
            stump.fit(X, residuals)
            self.models.append(stump)
            f_current += self.learning_rate * stump.predict(X)
        return self

    def predict_proba(self, X):
        raw_preds = np.ones(X.shape[0]) * self.f0
        for stump in self.models:
            raw_preds += self.learning_rate * stump.predict(X)
        return 1 / (1 + np.exp(-raw_preds))

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

# ----------------------------
# 4.d Обучение собственных моделей
# ----------------------------

## 4.d.1 Обучение собственного классификатора
cust_gb_clf = SimpleGBClassifier(n_estimators=100, learning_rate=0.1)
cust_gb_clf.fit(Xc_train, yc_train)
yc_pred_custom = cust_gb_clf.predict(Xc_test)

acc_cust = accuracy_score(yc_test, yc_pred_custom)
f1_cust = f1_score(yc_test, yc_pred_custom)

print("\n[Custom GB Classifier]")
print(f"Accuracy: {acc_cust:.4f}")
print(f"F1-score: {f1_cust:.4f}")

## 4.d.2 Обучение собственного регрессора
cust_gb_reg = SimpleGBRegressor(n_estimators=100, learning_rate=0.1)
cust_gb_reg.fit(Xr_train, yr_train)
yr_pred_custom = cust_gb_reg.predict(Xr_test)

mse_cust = mean_squared_error(yr_test, yr_pred_custom)
r2_cust = r2_score(yr_test, yr_pred_custom)

print("\n[Custom GB Regressor]")
print(f"MSE: {mse_cust:.4f}")
print(f"R^2: {r2_cust:.4f}")

# ----------------------------
# 4.e Оценка качества собственных моделей и сравнение
# ----------------------------

print("\nСравнение моделей:")
print("\nGradientBoostingClassifier (sklearn):")
print(f"Accuracy:  {acc:.4f}, F1-score: {f1:.4f}")

print("Custom GB Classifier:")
print(f"Accuracy: {acc_cust:.4f}, F1-score: {f1_cust:.4f}")

print("\nGradientBoostingRegressor (sklearn):")
print(f"MSE:  {mse:.4f}, R^2: {r2:.4f}")

print("Custom GB Regressor:")
print(f"MSE: {mse_cust:.4f}, R^2: {r2_cust:.4f}")

# ----------------------------
# 4.f Применение улучшений к собственным моделям
# ----------------------------

## Обучение улучшенных собственных моделей

### Улучшенный собственный классификатор
cust_gb_clf_improved = SimpleGBClassifier(n_estimators=200, learning_rate=0.1)
cust_gb_clf_improved.fit(Xc_train_improved, yc_train)
yc_pred_gb_improved = cust_gb_clf_improved.predict(Xc_test_improved)

acc_gb_imp = accuracy_score(yc_test, yc_pred_gb_improved)
f1_gb_imp = f1_score(yc_test, yc_pred_gb_improved)

print("\n[Custom GB Classifier, улучшенные данные]")
print(f"Accuracy: {acc_gb_imp:.4f}")
print(f"F1-score: {f1_gb_imp:.4f}")

### Улучшенный собственный регрессор
cust_gb_reg_improved = SimpleGBRegressor(n_estimators=200, learning_rate=0.1)
cust_gb_reg_improved.fit(Xr_train_improved, yr_train_improved)
yr_pred_gb_improved = cust_gb_reg_improved.predict(Xr_test_improved)

mse_gb_imp = mean_squared_error(yr_test_improved, yr_pred_gb_improved)
r2_gb_imp = r2_score(yr_test_improved, yr_pred_gb_improved)

print("\n[Custom GB Regressor, улучшенные данные]")
print(f"MSE:  {mse_gb_imp:.4f}")
print(f"R^2:  {r2_gb_imp:.4f}")



Breast Cancer Dataset shape: (569, 30)
Классы меток: [0 1]

Структура concrete_compressive_strength.variables:
                            name     role        type demographic description  \
0                         Cement  Feature  Continuous        None        None   
1             Blast Furnace Slag  Feature     Integer        None        None   
2                        Fly Ash  Feature  Continuous        None        None   
3                          Water  Feature  Continuous        None        None   
4               Superplasticizer  Feature  Continuous        None        None   
5               Coarse Aggregate  Feature  Continuous        None        None   
6                 Fine Aggregate  Feature  Continuous        None        None   
7                            Age  Feature     Integer        None        None   
8  Concrete compressive strength   Target  Continuous        None        None   

    units missing_values  
0  kg/m^3             no  
1  kg/m^3             no

Выводы:

1. **Бейзлайн модели** (GradientBoostingClassifier и GradientBoostingRegressor из sklearn) показали высокие показатели качества на обоих датасетах.
2. **Улучшение моделей** путём отбора признаков для классификации и создания новых признаков для регрессии позволило повысить качество моделей.
3. **Собственные реализации градиентного бустинга** (SimpleGBClassifier и SimpleGBRegressor) продемонстрировали основную логику бустинга, но уступили готовым реализациям из sklearn по качеству.
4. **Применение улучшений** к собственным моделям также позитивно сказалось на их показателях, но разрыв в качестве по-прежнему оставался значительным.
5. **Заключение**: Использование оптимизированных библиотечных реализаций градиентного бустинга предпочтительнее для практических задач, однако реализация собственных алгоритмов способствует лучшему пониманию принципов работы бустинга

